In [1]:
import warnings
warnings.filterwarnings("ignore")


import pandas as pd
import numpy as np

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score
from sklearn.metrics import auc as calculate_auc
from sklearn.datasets import load_breast_cancer

import matplotlib.pyplot as plt
import seaborn as sns

from aggmap import AggMap, AggModel

np.random.seed(666) #just for reaptable results

def prc_auc_score(y_true, y_score):
    precision, recall, threshold  = precision_recall_curve(y_true, y_score) #PRC_AUC
    auc = calculate_auc(recall, precision)
    return auc

In [2]:
data = load_breast_cancer()
dfx = pd.DataFrame(data.data, columns= data.feature_names)
y = pd.Series(data.target).map({0:'malignant', 1:'benign'})
dfy = pd.get_dummies(y) 
X = dfx.values.astype(float)
Y = dfy.values.astype(float)

In [3]:
mp = AggMap(dfx, metric = 'correlation')
mp.fit(cluster_channels = 5, verbose = 0)

2020-10-17 12:07:28,170 - INFO - [bidd-aggmap] - Calculating distance ...
2020-10-17 12:07:28,171 - INFO - [bidd-aggmap] - the number of process is 16


100%|##########| 30/30 [00:00<00:00, 558.40it/s]


2020-10-17 12:07:28,671 - INFO - [bidd-aggmap] - applying hierarchical clustering to obtain group information ...
2020-10-17 12:07:32,583 - INFO - [bidd-aggmap] - Applying grid feature map(assignment), this may take several minutes(1~30 min)
2020-10-17 12:07:32,593 - INFO - [bidd-aggmap] - Finished


## 10-fold cross validation and repeat 5 times 

In [ ]:
outer_fold = 10

run_all = []
for repeat_seed in [8, 16, 32, 64, 128]: #5 repeats random seeds
    
    outer = KFold(n_splits = outer_fold, shuffle = True, random_state = repeat_seed)
    outer_idx = outer.split(range(len(Y)), Y[:,0])

    for i, idx in enumerate(outer_idx):
        
        fold_num = "fold_%s" % str(i).zfill(2) 
        
        train_idx, test_idx = idx
        
        testY = Y[test_idx]
        testx = X[test_idx]
        
        trainx = X[train_idx]
        trainY = Y[train_idx]

        trainX = mp.batch_transform(trainx, scale_method = 'standard')
        testX = mp.batch_transform(testx, scale_method = 'standard')
        
        print("\n input train and test X shape is %s, %s " % (trainX.shape,  testX.shape))
        
        clf = AggModel.MultiClassEstimator(epochs = 200, gpuid = 1, batch_size = 8, verbose = 0)
        clf.fit(trainX, trainY)

        y_true = testY[:,0]
        y_pred = clf.predict(testX)[:,0]
        y_score = clf.predict_proba(testX)[:,0]

        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        
        acc = (tp + tn) / sum([tn, fp, fn, tp])
        
        sensitivity = tp / sum([tp, fn])
        specificity = tn / sum([tn, fp])
        
        prc_auc = prc_auc_score(y_true, y_score)
        roc_auc = roc_auc_score(y_true, y_score)
        
        precision = tp / sum([tp, fp])
        recall =  tp / sum([tp, fn]) #equals to sensitivity
        
        
        res = {'fold': fold_num,
               'repeat_seed':repeat_seed,
               
               'accuracy':acc, 
               
               'prc_auc':prc_auc, 
               'roc_auc':roc_auc,

               'sensitivity': sensitivity, 
               'specificity': specificity,
               
               'precision':precision,
               'recall':recall,
              }
        
        run_all.append(res)

100%|##########| 57/57 [00:00<00:00, 933.93it/s]



 input train and test X shape is (512, 6, 5, 5), (57, 6, 5, 5) 
MultiClassEstimator(batch_size=8, gpuid='1')


100%|##########| 57/57 [00:00<00:00, 700.66it/s]



 input train and test X shape is (512, 6, 5, 5), (57, 6, 5, 5) 
MultiClassEstimator(batch_size=8, gpuid='1')


100%|##########| 57/57 [00:00<00:00, 501.56it/s]


...


 input train and test X shape is (512, 6, 5, 5), (57, 6, 5, 5) 
MultiClassEstimator(batch_size=8, gpuid='1')


100%|##########| 57/57 [00:00<00:00, 832.04it/s]



 input train and test X shape is (512, 6, 5, 5), (57, 6, 5, 5) 
MultiClassEstimator(batch_size=8, gpuid='1')


100%|##########| 57/57 [00:00<00:00, 798.84it/s]



 input train and test X shape is (512, 6, 5, 5), (57, 6, 5, 5) 
MultiClassEstimator(batch_size=8, gpuid='1')


In [ ]:
clf._model.count_params()

In [ ]:
df = pd.DataFrame(run_all)
df.to_excel('results.xlsx')
df

In [ ]:
results = df.groupby(['repeat_seed']).apply(np.mean)
results

In [ ]:
results.mean().round(3)

In [31]:
results.mean().round(3)

repeat_seed    49.600
accuracy        0.974
prc_auc         0.997
roc_auc         0.995
sensitivity     0.984
specificity     0.957
precision       0.976
recall          0.984
dtype: float64

In [32]:
results.std().round(3)

repeat_seed    48.793
accuracy        0.002
prc_auc         0.000
roc_auc         0.001
sensitivity     0.002
specificity     0.009
precision       0.003
recall          0.002
dtype: float64